In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# パラメタ
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# ネットワークパラメタ
n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = {
    "out": tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    "out": tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
def RNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_steps, 0)

    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter:", str(step*batch_size), "Loss=", loss, "Acc=", acc
        step += 1
    print "Optimization Finish"
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Acc:", sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter: 1280 Loss= 1.96663 Acc= 0.289062
Iter: 2560 Loss= 1.74654 Acc= 0.429688
Iter: 3840 Loss= 1.38901 Acc= 0.523438
Iter: 5120 Loss= 1.11363 Acc= 0.59375
Iter: 6400 Loss= 0.957638 Acc= 0.65625
Iter: 7680 Loss= 1.21766 Acc= 0.570312
Iter: 8960 Loss= 1.02599 Acc= 0.648438
Iter: 10240 Loss= 0.836213 Acc= 0.773438
Iter: 11520 Loss= 0.613814 Acc= 0.804688
Iter: 12800 Loss= 0.87022 Acc= 0.671875
Iter: 14080 Loss= 0.726113 Acc= 0.773438
Iter: 15360 Loss= 0.437053 Acc= 0.835938
Iter: 16640 Loss= 0.556305 Acc= 0.859375
Iter: 17920 Loss= 0.337103 Acc= 0.890625
Iter: 19200 Loss= 0.358026 Acc= 0.882812
Iter: 20480 Loss= 0.238466 Acc= 0.945312
Iter: 21760 Loss= 0.507527 Acc= 0.828125
Iter: 23040 Loss= 0.214801 Acc= 0.921875
Iter: 24320 Loss= 0.451977 Acc= 0.835938
Iter: 25600 Loss= 0.42834 Acc= 0.882812
Iter: 26880 Loss= 0.257396 Acc= 0.914062
Iter: 28160 Loss= 0.315402 Acc= 0.90625
Iter: 29440 Loss= 0.37045 Acc= 0.890625
Iter: 30720 Loss= 0.355233 Acc= 0.867188
Iter: 32000 Loss= 0.221848 Acc= 0.9